In [1]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                       

In [257]:
!pip install geopandas
import geopandas
import matplotlib.pyplot as plt

     |████████████████████████████████| 931kB 23.6MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 25.7MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 41.7MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 19.6MB/s eta 0:00:01


### Gathering location data for all Denmark 


In [342]:
zones = geopandas.GeoDataFrame.from_file("https://raw.githubusercontent.com/ilzebormane/Coursera_capstone/master/copenhagen_districts.geojson")
# copy poly to new GeoDataFrame
zones["x"] = zones.centroid.map(lambda p: p.x)
zones["y"] = zones.centroid.map(lambda p: p.y)
points = zones[["name", "x", "y"]].copy()
zones.head(12)

,cartodb_id,id,lokaludval,price,k_price,offices,malls,supermark,total_pop,name,v_st_pl,v_st_op,v_kw_chrg,v_km_avg,v_prof_avg,total_ev_s,all_pop,geometry,x,y
0,1,17,3,26911,0.688110,24,0,26,58235,Osterbro,5,6,12000,10200,900,4,557673,"MULTIPOLYGON (((12.59777 55.70348, 12.59700 55...",12.586527,55.712243
1,2,23,7,19136,0.204380,11,0,18,47539,Valby,2,7,20000,17000,870,3,557673,"MULTIPOLYGON (((12.49337 55.67256, 12.49461 55...",12.502977,55.656698
2,3,100,6,26370,0.654452,11,0,9,11084,Kongens Enghave,0,3,7000,5950,700,12,557673,"MULTIPOLYGON (((12.53043 55.66327, 12.53179 55...",12.536129,55.648360
3,4,26,11,22026,0.384185,11,0,28,63794,Amager Ost,3,6,9000,7650,680,23,557673,"MULTIPOLYGON (((12.62850 55.68901, 12.62854 55...",12.625562,55.663357
4,5,21,9,15851,0.000000,8,0,12,44404,Bronshoj-Husum,3,8,23000,19550,765,6,557673,"MULTIPOLYGON (((12.46894 55.69976, 12.46698 55...",12.486300,55.710064
5,6,19,4,22972,0.443041,23,0,25,48240,Norrebro,3,0,0,0,608,21,557673,"MULTIPOLYGON (((12.53489 55.69439, 12.53482 55...",12.550974,55.696187
6,7,102,5,28138,0.764450,20,8,20,25046,Vesterbro,5,2,6000,5100,879,43,557673,"MULTIPOLYGON (((12.53179 55.66324, 12.53059 55...",12.551553,55.664883
7,8,105,2,29669,0.859703,114,1,31,55766,Indre By,12,6,17000,14450,960,54,557673,"MULTIPOLYGON (((12.60024 55.69515, 12.60027 55...",12.578535,55.683996
8,11,20,10,19625,0.234804,13,0,14,46496,Bispebjerg,5,7,26000,22100,875,11,557673,"MULTIPOLYGON (((12.51179 55.70840, 12.51211 55...",12.533830,55.714078
9,12,106,1,31924,1.000000,14,0,5,11760,Christianshavn,2,2,3000,2550,687,43,557673,"MULTIPOLYGON (((12.62482 55.69014, 12.62516 55...",12.606506,55.682234


In [309]:
cph_data = points
cph_data = cph_data.rename(columns= {"name" : "Area", "x":"Longitude","y":"Latitude"})
cph_data = cph_data[["Area", "Latitude", "Longitude"]]
cph_data.shape

(13, 3)

In [310]:
cph_data

,Area,Latitude,Longitude
0,Osterbro,55.712243,12.586527
1,Valby,55.656698,12.502977
2,Kongens Enghave,55.648360,12.536129
3,Amager Ost,55.663357,12.625562
4,Bronshoj-Husum,55.710064,12.486300
5,Norrebro,55.696187,12.550974
6,Vesterbro,55.664883,12.551553
7,Indre By,55.683996,12.578535
8,Bispebjerg,55.714078,12.533830
9,Christianshavn,55.682234,12.606506


In [311]:
address = 'Copenhagen, DK'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Copenhagen are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Copenhagen are 55.6867243, 12.5700724.


### Defining FS Credidentials

In [312]:

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OR33FW3CFIQYODQ5DQ4250ESWHZASDXVZMUC1LLWTZL3PVAG
CLIENT_SECRET:MROIDHEFTEXCTVSHTDLFJ3WPA5XPQSSUDJ5W4XV5P1V43KRG


### Gathering nearby Vegetarian / Vegan restaurants per each postal area (within a radius of 500m)

In [313]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [317]:
LIMIT = 100
radius = 3000
categoryId = '4bf58dd8d48988d1d3941735' #Category for Vegetarian and Vegan restaurants

In [330]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        print(name)    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location'],
            v['venue']['location']['distance'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Address',
                  'Distnace',           
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [331]:
dt_venues = getNearbyVenues(names=cph_data['Area'],
                                  latitudes=cph_data['Latitude'],
                                   longitudes=cph_data['Longitude']
                                )
dt_venues

Osterbro
Valby
Kongens Enghave
Amager Ost
Bronshoj-Husum
Norrebro
Vesterbro
Indre By
Bispebjerg
Christianshavn
Fredriksberg
Amager Vest
Vanlose


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Address,Distnace,Venue Latitude,Venue Longitude,Venue Category
0,Osterbro,55.712243,12.586527,Souls,"{'address': 'Melchiors Plads 3', 'lat': 55.703...",976,55.703503,12.585135,Vegetarian / Vegan Restaurant
1,Osterbro,55.712243,12.586527,Nordisk Falafel,"{'address': 'Nordre Frihavnsgade 55', 'lat': 5...",1083,55.702630,12.583862,Vegetarian / Vegan Restaurant
2,Osterbro,55.712243,12.586527,Kolonial 2100,"{'address': 'Landskronagade 27', 'lat': 55.713...",981,55.713032,12.570944,Vegetarian / Vegan Restaurant
3,Osterbro,55.712243,12.586527,42 Raw Waterfront,"{'address': 'Tuborg Havnevej 4', 'lat': 55.725...",1555,55.725686,12.579755,Vegetarian / Vegan Restaurant
4,Osterbro,55.712243,12.586527,Cafe Baryl,"{'address': 'Århusgade 78', 'lat': 55.706445, ...",673,55.706445,12.583466,Café
5,Osterbro,55.712243,12.586527,Veggie Heroes,"{'address': 'Blegdamsvej 106', 'lat': 55.69734...",1849,55.697345,12.573469,Vegetarian / Vegan Restaurant
6,Osterbro,55.712243,12.586527,Halifax,"{'address': 'Trianglen 1', 'crossStreet': 'Øst...",1556,55.699150,12.577797,Burger Joint
7,Valby,55.656698,12.502977,Cafe On The Six,"{'address': 'Tingstedet 6', 'lat': 55.66574, '...",1236,55.665740,12.514419,Vegetarian / Vegan Restaurant
8,Valby,55.656698,12.502977,G64,"{'address': 'Grønttorvet 64', 'lat': 55.655656...",248,55.655656,12.506470,Vegetarian / Vegan Restaurant
9,Valby,55.656698,12.502977,Jordbær i haven,"{'lat': 55.66549862153517, 'lng': 12.491299558...",1223,55.665499,12.491300,Vegetarian / Vegan Restaurant


In [332]:
cphs_map = folium.Map(location=[latitude, longitude],
                     tiles = 'cartodbpositron',
                    width=700, height=500,
                    zoom_start = 12)

for i in range(0, len(dt_venues)):
    name = dt_venues.iloc[i]['Neighborhood']
    folium.Marker([dt_venues.iloc[i]["Neighborhood Latitude"],dt_venues.iloc[i]["Neighborhood Longitude"]],
                 popup = name).add_to(cphs_map)
cphs_map

In [320]:
LIMITall =1000
radiusall = 25000
###Getting venues of CPH
venues_listall = []
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radiusall, 
            LIMITall,
            categoryId)

# make the GET request
resultsall = requests.get(url).json()["response"]['groups'][0]['items']

venues_listall.append([(
            "Copenhagen", 
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['location'],
            v['venue']['location']['distance'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in resultsall])
    
cph_venues = pd.DataFrame([item for venue_listall in venues_listall for item in venue_listall])
cph_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Address',
                  'Distnace',           
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']


In [321]:
cph_venues.shape

(55, 9)

### Investigation of data and venues

Data currently shows

In [322]:
cphu_venues = cph_venues[['Venue', 'Venue Latitude', 'Venue Longitude' ]]
cphu_venues = cphu_venues.drop_duplicates(subset=None, keep='first')
cphu_venues =cphu_venues.reset_index()
cphu_venues = cphu_venues[['Venue', 'Venue Latitude', 'Venue Longitude' ]]
cphu_venues['Venue'] = cphu_venues['Venue'].str.replace(r"[']", '')
cphu_venues.shape

(55, 3)

In [323]:
cph_venues.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Address,Distnace,Venue Latitude,Venue Longitude,Venue Category
0,Copenhagen,55.686724,12.570072,Morgenstedet,"{'address': 'Christiania', 'lat': 55.673893930...",2521,55.673894,12.603187,Vegetarian / Vegan Restaurant
1,Copenhagen,55.686724,12.570072,Plant Power Food,"{'address': 'Fælledvej 15', 'lat': 55.68945464...",692,55.689455,12.560152,Vegetarian / Vegan Restaurant
2,Copenhagen,55.686724,12.570072,Souls,"{'address': 'Melchiors Plads 3', 'lat': 55.703...",2093,55.703503,12.585135,Vegetarian / Vegan Restaurant
3,Copenhagen,55.686724,12.570072,Kaf,"{'address': 'Birkegade 21', 'crossStreet': 'Eg...",884,55.690880,12.558053,Vegetarian / Vegan Restaurant
4,Copenhagen,55.686724,12.570072,Falafel Factory,"{'address': 'Nørrebrogade 63', 'lat': 55.69043...",1048,55.690432,12.554716,Vegetarian / Vegan Restaurant
5,Copenhagen,55.686724,12.570072,Souls Nørreport,"{'address': '63 Nørre Farimagsgade', 'lat': 55...",202,55.685178,12.568380,Vegetarian / Vegan Restaurant
6,Copenhagen,55.686724,12.570072,Riz Raz,"{'address': 'Kompagnistræde 20', 'lat': 55.677...",1123,55.677075,12.575304,Vegetarian / Vegan Restaurant
7,Copenhagen,55.686724,12.570072,Nordisk Falafel,"{'address': 'Nordre Frihavnsgade 55', 'lat': 5...",1970,55.702630,12.583862,Vegetarian / Vegan Restaurant
8,Copenhagen,55.686724,12.570072,Vita Boost,"{'address': 'Torvehallerne', 'lat': 55.6838051...",325,55.683805,12.569912,Food & Drink Shop
9,Copenhagen,55.686724,12.570072,42 Raw,"{'address': 'Pilestræde 32', 'lat': 55.6806736...",935,55.680674,12.580412,Vegetarian / Vegan Restaurant


In [343]:
acces = dt_venues.groupby('Neighborhood').count()
acces = acces[["Venue"]]
acces["name"] = acces.index
acces

,Venue,name
Neighborhood,,
Amager Ost,3,Amager Ost
Bispebjerg,4,Bispebjerg
Bronshoj-Husum,1,Bronshoj-Husum
Christianshavn,8,Christianshavn
Fredriksberg,6,Fredriksberg
Indre By,38,Indre By
Norrebro,28,Norrebro
Osterbro,7,Osterbro
Valby,3,Valby


### Creating map of Copenhagen

In [372]:
cph_map = folium.Map(location=[latitude, longitude],
                     tiles = 'cartodbpositron',
                    width=700, height=500,
                    zoom_start = 12)

locationlist = cphu_venues[["Venue Latitude","Venue Longitude"]].values.tolist()
labels = cphu_venues["Venue"].values.tolist()

for i in range(0, len(cphu_venues)):
    name = cphu_venues.iloc[i]['Venue']
    folium.Marker([cphu_venues.iloc[i]["Venue Latitude"],cphu_venues.iloc[i]["Venue Longitude"]],
                 popup = name).add_to(cph_map)

polies = 'https://raw.githubusercontent.com/ilzebormane/Coursera_capstone/master/copenhagen_districts.geojson'
folium.GeoJson(polies).add_to(cph_map)


In [373]:
cph_map

In [374]:
cph_map.choropleth(geo_data=zones,
             data=acces, # my dataset
             columns=['name', 'Venue'], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.name', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='#Vegetarian/Vegan friendly restaurants within 3km radius')

In [375]:
cph_map

In [356]:
def getNearbyVenuesall(names, latitudes, longitudes, radius=2000):
    
    venues_lista=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        print(name)    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
            
        # make the GET request
        resultsa = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_lista.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location'],
            v['venue']['location']['distance'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in resultsa])

    nearby_venuesa = pd.DataFrame([item for venue_lista in venues_lista for item in venue_lista])
    nearby_venuesa.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Address',
                  'Distnace',           
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venuesa)

In [357]:
dta_venues = getNearbyVenuesall(names=cph_data['Area'],
                                  latitudes=cph_data['Latitude'],
                                   longitudes=cph_data['Longitude']
                                )
dta_venues

Osterbro
Valby
Kongens Enghave
Amager Ost
Bronshoj-Husum
Norrebro
Vesterbro
Indre By
Bispebjerg
Christianshavn
Fredriksberg
Amager Vest
Vanlose


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Address,Distnace,Venue Latitude,Venue Longitude,Venue Category
0,Osterbro,55.712243,12.586527,Østre Gasværk Teater,"{'address': 'Nyborggade 17', 'lat': 55.7098257...",397,55.709826,12.581865,Theater
1,Osterbro,55.712243,12.586527,Svanemøllehavnen,"{'address': 'Strandvænget 43', 'lat': 55.71632...",455,55.716327,12.585963,Harbor / Marina
2,Osterbro,55.712243,12.586527,Juno The Bakery,"{'address': 'Århusgade 48', 'lat': 55.706164, ...",743,55.706164,12.581619,Bakery
3,Osterbro,55.712243,12.586527,SATS,"{'address': 'Nygårdsvej 5', 'lat': 55.71218921...",644,55.712189,12.576246,Gym / Fitness Center
4,Osterbro,55.712243,12.586527,Konditaget Lüders,"{'lat': 55.70758799258327, 'lng': 12.598499146...",912,55.707588,12.598499,Outdoor Gym
5,Osterbro,55.712243,12.586527,Charlottehaven Health Club,"{'address': 'Hjørringgade 12C', 'lat': 55.7054...",768,55.705436,12.588549,Gym
6,Osterbro,55.712243,12.586527,At The Counter,"{'address': 'Randersgade 45', 'lat': 55.705136...",870,55.705136,12.580746,Coffee Shop
7,Osterbro,55.712243,12.586527,Souls,"{'address': 'Melchiors Plads 3', 'lat': 55.703...",976,55.703503,12.585135,Vegetarian / Vegan Restaurant
8,Osterbro,55.712243,12.586527,Brdr. D's Vinhandel,"{'address': 'Østerbrogade 152', 'lat': 55.7086...",701,55.708623,12.577372,Wine Shop
9,Osterbro,55.712243,12.586527,Svanemøllebugtens Vinterbadelaug,"{'address': 'Svaneknoppen 7', 'lat': 55.718033...",654,55.718033,12.588349,Bathing Area


In [358]:
# one hot encoding
dta_venues.head()
dt_onehot = pd.get_dummies(dta_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighborhood'] = dta_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]
dt_onehot.shape

(1159, 201)

In [359]:
dt_grouped = dt_onehot.groupby('Neighborhood').mean().reset_index()
dt_grouped

,Neighborhood,Zoo Exhibit,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Track,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo
0,Amager Ost,0.000000,0.000000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.02,0.000000,0.020000,0.000000,0.01,0.00
1,Amager Vest,0.000000,0.000000,0.00,0.013699,0.00,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
2,Bispebjerg,0.000000,0.010638,0.00,0.000000,0.00,0.000000,0.00,0.00,0.010638,...,0.010638,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010638,0.00,0.00
3,Bronshoj-Husum,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,...,0.000000,0.035714,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
4,Christianshavn,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.02,0.01,0.000000,...,0.000000,0.000000,0.000000,0.01,0.01,0.010000,0.030000,0.010000,0.00,0.00
5,Fredriksberg,0.050000,0.000000,0.00,0.000000,0.00,0.010000,0.01,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.030000,0.010000,0.00,0.01
6,Indre By,0.000000,0.000000,0.00,0.000000,0.00,0.010000,0.02,0.01,0.000000,...,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.030000,0.000000,0.00,0.00
7,Kongens Enghave,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,...,0.000000,0.034884,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00
8,Norrebro,0.000000,0.010000,0.01,0.000000,0.00,0.000000,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.00,0.01,0.010000,0.040000,0.020000,0.02,0.00
9,Osterbro,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,...,0.010000,0.010000,0.000000,0.00,0.02,0.000000,0.010000,0.010000,0.00,0.00


In [360]:
from sklearn.cluster import KMeans
num_top_venues = 5

for hood in dt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amager Ost----
         venue  freq
0  Coffee Shop  0.08
1        Beach  0.07
2  Pizza Place  0.07
3         Café  0.07
4       Bakery  0.07


----Amager Vest----
                 venue  freq
0           Restaurant  0.07
1          Supermarket  0.05
2  Sporting Goods Shop  0.04
3          Coffee Shop  0.04
4                 Café  0.04


----Bispebjerg----
                  venue  freq
0         Grocery Store  0.10
1                  Café  0.07
2  Gym / Fitness Center  0.06
3           Pizza Place  0.05
4                 Plaza  0.05


----Bronshoj-Husum----
                  venue  freq
0         Grocery Store  0.12
1           Supermarket  0.07
2                  Park  0.05
3                 Plaza  0.05
4  Gym / Fitness Center  0.05


----Christianshavn----
                     venue  freq
0  Scandinavian Restaurant  0.11
1               Restaurant  0.08
2                    Hotel  0.05
3              Coffee Shop  0.05
4                     Park  0.04


----Fredriksberg----
       

In [361]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18]

array([0, 2, 1, 4, 3, 3, 3, 1, 3, 0, 1, 4, 3], dtype=int32)

In [362]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [363]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_grouped['Neighborhood']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amager Ost,Coffee Shop,Pizza Place,Bakery,Beach,Café,Gym / Fitness Center,Sushi Restaurant,Supermarket,Burger Joint,Grocery Store
1,Amager Vest,Restaurant,Supermarket,Discount Store,Café,Sporting Goods Shop,Coffee Shop,Hotel,Bus Station,Golf Course,Park
2,Bispebjerg,Grocery Store,Café,Gym / Fitness Center,Pizza Place,Plaza,Gym,Park,Athletics & Sports,Bus Station,Scandinavian Restaurant
3,Bronshoj-Husum,Grocery Store,Supermarket,Plaza,Park,Gym / Fitness Center,Playground,Chinese Restaurant,Sushi Restaurant,Bakery,Pizza Place
4,Christianshavn,Scandinavian Restaurant,Restaurant,Hotel,Coffee Shop,Park,Café,Wine Bar,Bar,French Restaurant,Steakhouse
5,Fredriksberg,Park,Coffee Shop,Bakery,Zoo Exhibit,Café,Pizza Place,Music Venue,Gym / Fitness Center,Scandinavian Restaurant,French Restaurant
6,Indre By,Coffee Shop,Scandinavian Restaurant,Café,Beer Bar,Restaurant,Burger Joint,Wine Bar,Furniture / Home Store,Bakery,Hotel
7,Kongens Enghave,Grocery Store,Café,Bakery,Hotel,Gym / Fitness Center,Pool,Train Station,Convenience Store,Office,Harbor / Marina
8,Norrebro,Coffee Shop,Café,Beer Bar,Wine Bar,Pizza Place,Playground,Bakery,Cocktail Bar,Scandinavian Restaurant,Breakfast Spot
9,Osterbro,Coffee Shop,Bakery,Playground,Ice Cream Shop,Gym / Fitness Center,Supermarket,Sushi Restaurant,Park,Café,Pizza Place


In [369]:
sorted_cph =cph_data.sort_values(by=['Area'], ascending = True)
dt_merged = sorted_cph.rename(columns ={"Area":"Neighborhood"})
# add clustering labels
dt_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
dt_merged = dt_merged.rename(columns ={"Neighborhood": "name"})
dt_merged.head(12) # check the last columns!

,name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Amager Ost,55.663357,12.625562,0,Coffee Shop,Pizza Place,Bakery,Beach,Café,Gym / Fitness Center,Sushi Restaurant,Supermarket,Burger Joint,Grocery Store
11,Amager Vest,55.638721,12.572178,2,Restaurant,Supermarket,Discount Store,Café,Sporting Goods Shop,Coffee Shop,Hotel,Bus Station,Golf Course,Park
8,Bispebjerg,55.714078,12.533830,1,Grocery Store,Café,Gym / Fitness Center,Pizza Place,Plaza,Gym,Park,Athletics & Sports,Bus Station,Scandinavian Restaurant
4,Bronshoj-Husum,55.710064,12.486300,4,Grocery Store,Supermarket,Plaza,Park,Gym / Fitness Center,Playground,Chinese Restaurant,Sushi Restaurant,Bakery,Pizza Place
9,Christianshavn,55.682234,12.606506,3,Scandinavian Restaurant,Restaurant,Hotel,Coffee Shop,Park,Café,Wine Bar,Bar,French Restaurant,Steakhouse
10,Fredriksberg,55.679982,12.523552,3,Park,Coffee Shop,Bakery,Zoo Exhibit,Café,Pizza Place,Music Venue,Gym / Fitness Center,Scandinavian Restaurant,French Restaurant
7,Indre By,55.683996,12.578535,3,Coffee Shop,Scandinavian Restaurant,Café,Beer Bar,Restaurant,Burger Joint,Wine Bar,Furniture / Home Store,Bakery,Hotel
2,Kongens Enghave,55.648360,12.536129,1,Grocery Store,Café,Bakery,Hotel,Gym / Fitness Center,Pool,Train Station,Convenience Store,Office,Harbor / Marina
5,Norrebro,55.696187,12.550974,3,Coffee Shop,Café,Beer Bar,Wine Bar,Pizza Place,Playground,Bakery,Cocktail Bar,Scandinavian Restaurant,Breakfast Spot
0,Osterbro,55.712243,12.586527,0,Coffee Shop,Bakery,Playground,Ice Cream Shop,Gym / Fitness Center,Supermarket,Sushi Restaurant,Park,Café,Pizza Place


In [370]:
cluster_map = folium.Map(location=[latitude, longitude],
                     tiles = 'cartodbpositron',
                    width=700, height=500,
                    zoom_start = 12)

locationlist = cphu_venues[["Venue Latitude","Venue Longitude"]].values.tolist()
labels = cphu_venues["Venue"].values.tolist()

for i in range(0, len(cphu_venues)):
    name = cphu_venues.iloc[i]['Venue']
    folium.Marker([cphu_venues.iloc[i]["Venue Latitude"],cphu_venues.iloc[i]["Venue Longitude"]],
                 popup = name).add_to(cluster_map)

polies = 'https://raw.githubusercontent.com/ilzebormane/Coursera_capstone/master/copenhagen_districts.geojson'
folium.GeoJson(polies).add_to(cluster_map)

cluster_map.choropleth(geo_data=zones,
             data=dt_merged, # my dataset
             columns=['name', 'Cluster Labels'], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.name', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Clusters')
cluster_map